In [ ]:
#!/usr/bin/env python
# coding: utf-8

In[ ]:

minimal requirements<br>
python >= 3.6<br>
pandas >= 1

In [1]:
import numpy as np
import pandas as pd
from typing import Dict
from functools import lru_cache
from operator import itemgetter
from collections import Counter
import pyomo.environ as pe
import pyomo.opt as po
import math
from pyomo.environ import value

You have<br>
* a ribbon of length 'LENGTH' meters (the problem is 1D).<br>
* defects that lies on the ribbon. Each defect has<br>
    * a position `x`<br>
    * and a class [`a`, `b`, `c`, ...]<br>
* set of available products. Each product can be produced an infinite number of times, and has:<br>
    * a size (along the same dimension as the ribbon)<br>
    * a value (see next paragraph)<br>
    * and a threshold for the maximum number of defects of each size it can contains<br>
<br>
A solution is an affectation of products on the ribbon. An affectation, to be valid, need to be:<br>
* at __integer positions__<br>
* have no intersection between its products placement. If you place a product P1 of size 3, at position x=2, you cant have any product affected at positions `x=3` nor `x=4`.<br>
* each product placed on the ribbon needs to contain less (or equal) defects of each class from the ribbon than authorized by its thresholds. A product P1 of size 3 placed at `x=2`, contains all defects of the ribbon which `x` is in `[2, 5]`. If in this interval, we have 3 defects of class `a`, and that threshold of P1 authorized maximum 2 defects of class `a`, the affectation is invalid<br>
<br>
The value of the solution is the sum of the value of the individual products placed on it. Part of the ribbon with no product affected amount to 0<br>
<br>
<br>
Benchmark:<br>
* this notebook generates random instances.<br>
* if you run this cells in this order after the seeding (and without calling other randoms/re-executing cells), you can find a solution of value 358.

# Setup<br>
## we define some fixed parameters and generate some random instances

seed random for reproductibility of benchmark instance

In [2]:
np.random.seed(5)

length of the glass ribbon to cut

In [3]:
LENGTH = 100

## defects

In [4]:
n_defects = 20

classe 1, classe 2 etc

In [5]:
n_defect_classes = 4
assert n_defect_classes <= 26, "too much defects classes"

generates defects position

In [6]:
defects_x = np.random.uniform(0, LENGTH, (n_defects))

generates their classes

In [7]:
defects_class = np.random.choice(
    list('abcdefghijklmnopqrstuvwxyz'[:n_defect_classes]),
    (n_defects)
)

summarize

In [8]:
defects = pd.DataFrame(
    columns=['x', 'class'],
    data = np.array([defects_x, defects_class]).T
)
defects['x'] = defects['x'].astype(float)
defects = defects.sort_values(by=['x'])
defects = defects.reset_index(drop = True)
defects.index += 1

In [9]:
defects.head(3).style.set_caption('extract of defects of the ribbon').format('{:,.2f}', subset='x')

,x,class
1,8.07,a
2,15.83,b
3,18.77,a


## products

In [10]:
n_products = 4

In [11]:
class Product:
    def __init__(self):
        self.length: int = np.random.randint(4,10)
        self.value: int = np.random.randint(1,10)
        self.max_defects: Dict[int, int] = {
            key: np.random.randint(1,5) for key in defects_class
        }
    def __repr__(self):
        return f'Product of size {self.length}, value {self.value} and max_defects {self.max_defects}'

generate n_products random product (ie random size, value and threshold)

In [12]:
Products = [Product() for _ in range(n_products)]
print('\n * '.join([f'The {n_products} products are'] + [str(p) for p in Products]))

The 4 products are
 * Product of size 5, value 4 and max_defects {'c': 4, 'b': 1, 'd': 4, 'a': 3}
 * Product of size 9, value 2 and max_defects {'c': 4, 'b': 3, 'd': 1, 'a': 3}
 * Product of size 4, value 1 and max_defects {'c': 3, 'b': 1, 'd': 1, 'a': 2}
 * Product of size 7, value 6 and max_defects {'c': 1, 'b': 3, 'd': 1, 'a': 4}


In [13]:
M = pe.ConcreteModel()
# glpk
solver = po.SolverFactory('glpk')

sets

In [15]:
param_value = {}
param_length = {}
param_max_defects = {}
defects_param = {}
for index, product in enumerate(Products):
    param_value[index+1] = int(math.floor(product.value))
    param_length[index+1] = product.length
    for key,value in product.max_defects.items():
        param_max_defects[(index+1, key)] = value

In [16]:
alphabet = sorted({ _ for _ in list('abcdefghijklmnopqrstuvwxyz'[:n_defect_classes])})

In [17]:
for index, row in defects.iterrows():
    defects_param[index, row['class']] = 1

In [18]:
M.length = LENGTH
M.defects = n_defects
M.alphabet = pe.Set(initialize=alphabet)
M.product = pe.RangeSet(1, n_products)
M.defect = pe.RangeSet(1, n_defects)
M.product_value = pe.Param(M.product, initialize = param_value, default=0)
M.product_length = pe.Param(M.product, initialize = param_length, default=0)
M.product_max_defects = pe.Param(M.product, M.alphabet, initialize = param_max_defects, default=0)
M.defect_info = pe.Param(M.defect, M.alphabet, initialize = defects_param, default=0)

In [19]:
M.x = pe.Var(M.defect, M.product, domain=pe.Binary)

In [20]:
obj_expr = sum(M.product_value[p] * M.x[d,p] for p in M.product for d in M.defect)
M.obj = pe.Objective(sense=pe.minimize, expr=obj_expr)

In [21]:
M.notoverlapping = pe.ConstraintList()
for d in M.defect:
    lhs= sum(M.x[d,p] for p in M.product)
    rhs=1
    M.notoverlapping.add(lhs == rhs)

In [22]:
M.exceedlength = pe.Constraint(
expr =  sum(M.product_length[p]* M.x[d,p] for p in M.product for d in M.defect) <= M.length)

In [23]:
M.coveringalldefects = pe.Constraint(
expr =  sum(M.x[d,p] for d in M.defect for p in M.product) == M.defects)

In [24]:
M.respectthreshold = pe.ConstraintList()
for p in M.product:
    for c in list(alphabet):
        lhs= sum(M.x[d,p] * M.defect_info[d, c] for d in M.defect)
        rhs=M.product_max_defects[p,c]
        M.respectthreshold.add(lhs <= rhs)

In [25]:
results = solver.solve(M, tee = True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\bozorg\AppData\Local\Temp\tmpvci2vta9.glpk.raw --wglp C:\Users\bozorg\AppData\Local\Temp\tmpg0fy0bim.glpk.glp
 --cpxlp C:\Users\bozorg\AppData\Local\Temp\tmp712ph05t.pyomo.lp
Reading problem data from 'C:\Users\bozorg\AppData\Local\Temp\tmp712ph05t.pyomo.lp'...
C:\Users\bozorg\AppData\Local\Temp\tmp712ph05t.pyomo.lp:607: warning: lower bound of variable 'x1' redefined
C:\Users\bozorg\AppData\Local\Temp\tmp712ph05t.pyomo.lp:607: warning: upper bound of variable 'x1' redefined
39 rows, 81 columns, 321 non-zeros
80 integer variables, all of which are binary
687 lines were read
Writing problem data to 'C:\Users\bozorg\AppData\Local\Temp\tmpg0fy0bim.glpk.glp'...
564 lines were written
GLPK Integer Optimizer, v4.65
39 rows, 81 columns, 321 non-zeros
80 integer variables, all of which are binary
Preprocessing...
35 rows, 80 columns, 310 non-zeros
80 integer variables, all of which are binary
Scalin

In [26]:
if (results.solver.status == po.SolverStatus.ok) and (results.solver.termination_condition == po.TerminationCondition.optimal):
     print ("this is feasible and optimal")
     print(str(results.solver))
     M.display()
elif results.solver.termination_condition == po.TerminationCondition.infeasible:
     print ("do something about it? or exit?")
else:
     # something else is wrong
     print (str(results.solver))

this is feasible and optimal

- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 115
      Number of created subproblems: 115
  Error rc: 0
  Time: 0.05242156982421875

Model unknown

  Variables:
    x : Size=80, Index=x_index
        Key     : Lower : Value : Upper : Fixed : Stale : Domain
         (1, 1) :     0 :   0.0 :     1 : False : False : Binary
         (1, 2) :     0 :   0.0 :     1 : False : False : Binary
         (1, 3) :     0 :   1.0 :     1 : False : False : Binary
         (1, 4) :     0 :   0.0 :     1 : False : False : Binary
         (2, 1) :     0 :   0.0 :     1 : False : False : Binary
         (2, 2) :     0 :   1.0 :     1 : False : False : Binary
         (2, 3) :     0 :   0.0 :     1 : False : False : Binary
         (2, 4) :     0 :   0.0 :     1 : False : False : Binary
         (3, 1) :     0 :   0.0 :     1 : False : False : Binary
         (3, 2) :     0 :   0.0 :     1 : False : Fa

In [29]:
print('---------------------------')
#M.pprint()<br>
#M.obj.display()<br>
#M.defect_info.display()<br>
#print('maxDef')<br>
#M.product_max_defects.display()<br>
#print('vlaue')<br>
#M.product_value.display()<br>
#print('lenght')<br>
#M.product_length.display()<br>
#print('var')<br>
#M.x.display()<br>
#print(defects)<br>
#defect_info.display()

---------------------------


## Solution structure<br>
<br>
The Solution class needs to be instantiated with the current "defects" you are using.<br>
```python<br>
current_solution = Solution(defects)<br>
```<br>
<br>
You can create the Solution iteratively, by placing product `p` at position `position`, with the method<br>
```python<br>
current_solution.add_product(p, position)<br>
```<br>
<br>
You can compute your current solution score with<br>
```python<br>
current_solution.compute_value()<br>
```<br>
<br>
You can check if you dont have any invalidities with<br>
```python<br>
current_solution.checker()<br>
```

In [33]:
class PlacedProduct:
    """
    helper class representing a product placed on a position of the ribbon
    """
    def __init__(self, product: Product, position: int):
        self.product = product
        self.position = position

In [37]:
class Solution:
    def __init__(self, defects:pd.DataFrame):
        self.placedProducts = []
        self.defects = defects
    def add_product(self, product: Product, position: int=0)->None:
        self.placedProducts.append(PlacedProduct(product, position))
    def compute_value(self):
        return sum((pp.product.value for pp in self.placedProducts))
    def checker(self):
        bob = pd.DataFrame(
            np.array([[pp.position for pp in self.placedProducts], [pp.product.length for pp in self.placedProducts]]).T,
            columns=['pos', 'length']
        )
        bob = bob.sort_values('pos')
        a0 = bob[np.floor(bob['pos']) < bob['pos']]
        assert len(a0) == 0, f'placedProducts at non integer positions {*a0.to_list(), }'
        a1 = bob.loc[bob.sum(axis=1)>LENGTH, 'pos']
        assert len(a1) == 0, f'placedProducts exceed LENGTH {*a1.to_list(), }'
        a2 = bob.sort_values('pos')
        a2 = bob.loc[bob['pos'] + bob['length'] > bob['pos'].shift(-1), 'pos']
        assert len(a2)==0, f'overlapping placedProducts at positions {*a2.to_list(), }'
        # check max defects OK
        for pp in self.placedProducts:
            defects_in_plate = self.defects.loc[(self.defects['x'] >= pp.position) & (self.defects['x'] <= pp.position + pp.product.length), "class"].to_list()
            a3 = Counter(defects_in_plate)-Counter(pp.product.max_defects)
            assert not a3, f"plate at position {pp.position} contains too much defects of classes {*a3.keys(), }"
        print("solution valid")

 demo OK with initial seeding

In [38]:
sol = Solution(defects)
sol.add_product(
    product=Products[0],
    position=10
)
print('value', sol.compute_value())
sol.checker()

value 4
solution valid


 demo not OK cause overlap with previous plate

In [43]:
sol = Solution(defects)
sol.add_product(
    product=Products[3],
    position=0
)
print('value', sol.compute_value())
sol.checker()

value 6
solution valid


In [44]:
sol.add_product(
    product=Products[3],
    position=2
)
print('value', sol.compute_value())
sol.checker()

value 12


AssertionError: overlapping placedProducts at positions (0,)

 demo not OK because too much defects

In [46]:
sol = Solution(defects)
sol.add_product(
    product=Products[2],
    position=5
)
print('value', sol.compute_value())
sol.checker()

value 1
solution valid


## HELPERS<br>
<br>
Some functions that can be used build a solution

In [ ]:
from functools import lru_cache
from operator import itemgetter
from collections import Counter

helpers

In [ ]:
def defects_counts(x: float, length: float) -> Counter:
    """
    Count number of defects of each class on the ribbon [x, x+length]
    return {defect class: number of defects}
    """
    # filter index of defects within range
    res = defects.loc[(defects["x"] >= x) & (defects["x"] <= x+length), "class"].to_list()
    return Counter(res)

if we start at x, a plate of length 7 will have quality: number

In [ ]:
defects_counts(20, 7)

In [ ]:
def contains(container: Counter, content: Counter) -> bool:
    """
    check if all values of container are bigger than those of content.
    Can compare <defects_counts> to products thresholds
    """
    return not content - container

In [ ]:
print(contains(Counter([1,1]), Counter([1])))
print(contains(Counter([1,1]), Counter([1,2])))

In [ ]:
def possible(x: float, p: Product) -> bool:
    """
    return True product p is compatible with position x
    """
    defects_present = defects_counts(x, p.length)
    return contains(Counter(p.max_defects), defects_present)

In [ ]:
possible(1, Products[0])

example of solution creation from a list res = [(position first element, product first element), (position second element, product second element), ...]<br>
es = [(,), (,), ...]<br>
ol = Solution(defects)<br>
or item in res:<br>
   sol.add_product(item[1], item[0])<br>
rint(sol.compute_value())<br>
ol.checker()